In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v003_5'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class02/train0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 525 images belonging to 2 classes.
Found 141 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_xception_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7196\1152529500.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
65/65 [==============================] - 75s 1s/step - loss: 0.6025 - accuracy: 0.6925 - val_loss: 0.3783 - val_accuracy: 0.8088 - lr: 0.0010
Epoch 2/3
65/65 [==============================] - 69s 1s/step - loss: 0.5114 - accuracy: 0.7640 - val_loss: 0.4517 - val_accuracy: 0.7647 - lr: 0.0010
Epoch 3/3
65/65 [==============================] - 70s 1s/step - loss: 0.4561 - accuracy: 0.8046 - val_loss: 0.3753 - val_accuracy: 0.8162 - lr: 0.0010


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7196\81111685.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
65/65 [==============================] - 135s 2s/step - loss: 0.4671 - accuracy: 0.8046 - val_loss: 0.2851 - val_accuracy: 0.8603 - lr: 1.0000e-04
Epoch 2/10
65/65 [==============================] - 122s 2s/step - loss: 0.4318 - accuracy: 0.8221 - val_loss: 0.3473 - val_accuracy: 0.8309 - lr: 1.0000e-04
Epoch 3/10
65/65 [==============================] - 131s 2s/step - loss: 0.4052 - accuracy: 0.8201 - val_loss: 0.2687 - val_accuracy: 0.8971 - lr: 1.0000e-04
Epoch 4/10
65/65 [==============================] - 119s 2s/step - loss: 0.4115 - accuracy: 0.8375 - val_loss: 0.2697 - val_accuracy: 0.8971 - lr: 1.0000e-04
Epoch 5/10
65/65 [==============================] - 122s 2s/step - loss: 0.3458 - accuracy: 0.8511 - val_loss: 0.2917 - val_accuracy: 0.8456 - lr: 1.0000e-04
Epoch 6/10
65/65 [==============================] - 124s 2s/step - loss: 0.3533 - accuracy: 0.8491 - val_loss: 0.2600 - val_accuracy: 0.8971 - lr: 1.0000e-04
Epoch 7/10
65/65 [==============================] - 

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7196\2279829289.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


65/65 [==============================] - 279s 4s/step - loss: 0.3855 - accuracy: 0.8491 - val_loss: 0.2583 - val_accuracy: 0.9191 - lr: 1.0000e-05
Epoch 2/150
65/65 [==============================] - 243s 4s/step - loss: 0.3336 - accuracy: 0.8607 - val_loss: 0.2830 - val_accuracy: 0.9118 - lr: 1.0000e-05
Epoch 3/150
65/65 [==============================] - 243s 4s/step - loss: 0.3512 - accuracy: 0.8549 - val_loss: 0.2848 - val_accuracy: 0.9044 - lr: 1.0000e-05
Epoch 4/150
65/65 [==============================] - 251s 4s/step - loss: 0.3725 - accuracy: 0.8414 - val_loss: 0.2818 - val_accuracy: 0.9118 - lr: 1.0000e-05
Epoch 5/150
65/65 [==============================] - 248s 4s/step - loss: 0.3168 - accuracy: 0.8723 - val_loss: 0.2807 - val_accuracy: 0.8897 - lr: 1.0000e-05
Epoch 6/150
65/65 [==============================] - 250s 4s/step - loss: 0.3357 - accuracy: 0.8665 - val_loss: 0.2891 - val_accuracy: 0.9191 - lr: 1.0000e-05
Epoch 7/150
65/65 [==============================] - 250s 

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
spe=recall_score(valid_generator.classes, y_pred,pos_label=0)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 141 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7196\3759042783.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(valid_generator)


Confusion Matrix:
[[88  4]
 [ 8 41]]
Sensitivity:  0.8367346938775511
Specificity:  0.9565217391304348
Precision score: 0.9111111111111111
Accuracy: 0.9148936170212766
F1 score: 0.8723404255319148
MCC score: 0.8103332939743233
ROC AUC score: 0.9525288376220054


In [13]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[2.17503053e-03]
 [1.34638160e-01]
 [1.17332377e-02]
 [1.32733500e-02]
 [1.44673651e-02]
 [2.72588339e-03]
 [2.14461200e-02]
 [2.27300137e-01]
 [2.87674457e-01]
 [8.10224473e-01]
 [1.75688826e-02]
 [1.16572126e-04]
 [2.57822452e-03]
 [9.80978482e-04]
 [1.63742378e-02]
 [8.43540402e-05]
 [7.95805335e-01]
 [1.96874398e-03]
 [4.19863135e-01]
 [4.70412808e-04]
 [9.02422741e-02]
 [1.21487156e-02]
 [3.55480202e-02]
 [2.12858729e-02]
 [9.42707062e-02]
 [2.73215890e-01]
 [1.11124143e-02]
 [3.07782134e-03]
 [1.36337122e-02]
 [1.52562514e-01]
 [5.56324422e-03]
 [1.38446450e-01]
 [1.12680597e-02]
 [1.82951260e-02]
 [2.63724197e-03]
 [2.10265652e-03]
 [3.63616622e-03]
 [4.03233431e-02]
 [2.34147742e-01]
 [2.32708558e-01]
 [2.82495166e-03]
 [5.41568326e-04]
 [2.39433793e-04]
 [3.55980426e-01]
 [1.30145447e-04]
 [4.27382346e-03]
 [3.96612613e-03]
 [7.07283325e-04]
 [1.68495830e-02]
 [1.96860451e-02]
 [1.29655555e-01]
 [7.27675855e-04]
 [4.04224768e-02]
 [5.62128262e-04]
 [3.40430648e-04]
 [1.024065

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class02/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
spe=recall_score(test_generator.classes, y_pred,pos_label=0)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 169 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7196\3439002409.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


Confusion Matrix:
[[107   6]
 [ 24  32]]
Sensitivity:  0.5714285714285714
Specificity:  0.9469026548672567
Precision score: 0.8421052631578947
Accuracy: 0.8224852071005917
F1 score: 0.6808510638297872
MCC score: 0.5844039764442677
ROC AUC score: 0.8712073324905184


In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[7.65616745e-02]
 [9.23650190e-02]
 [3.03795218e-01]
 [9.73157212e-03]
 [1.36090266e-02]
 [1.33526733e-03]
 [8.43006298e-02]
 [1.11796334e-02]
 [5.12259593e-03]
 [2.21134326e-03]
 [4.81446367e-03]
 [3.80805954e-02]
 [1.93740204e-01]
 [2.69134849e-01]
 [2.42308201e-03]
 [5.19977533e-04]
 [3.77215475e-01]
 [6.53268456e-01]
 [7.64237018e-04]
 [2.39781410e-01]
 [4.92360890e-01]
 [4.32445522e-04]
 [7.92151131e-03]
 [2.34744996e-01]
 [9.80206183e-04]
 [3.53556522e-03]
 [2.26468258e-02]
 [7.24978149e-02]
 [4.92465794e-01]
 [1.43321440e-01]
 [1.46713946e-03]
 [8.57099891e-02]
 [5.49576012e-03]
 [7.07944483e-02]
 [1.99004382e-01]
 [3.36757419e-03]
 [8.92341882e-02]
 [2.59805266e-02]
 [2.13184990e-02]
 [7.06574261e-01]
 [6.39908612e-02]
 [5.99304885e-02]
 [6.67947391e-03]
 [1.66514528e-03]
 [1.17597980e-02]
 [5.84826386e-03]
 [8.50627257e-04]
 [4.37112013e-03]
 [2.37469003e-01]
 [2.10135104e-03]
 [3.50445174e-02]
 [1.63253292e-01]
 [5.43926551e-04]
 [1.84322684e-03]
 [2.68567121e-03]
 [5.248450